<a href="https://colab.research.google.com/github/mozzaquatro/financial_market/blob/main/21_AF_Magic%20Formula%20de%20Joel%20Greenblatt%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analise - Magic Formula de Joel Greenblatt 
1. importando bibliotecas

In [2]:
import numpy as np
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')

import requests

2. Obtendo e tratando os dados

In [3]:
url = 'http://www.fundamentus.com.br/resultado.php'

In [4]:
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)

In [5]:
df = pd.read_html(r.text,  decimal=',', thousands='.')[0]

In [6]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,PMET3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","4,10%",0.0,-2.908630e+08,0.00,"37,74%"
1,CLAN3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","-1,05%",0.0,1.012240e+09,0.00,"-63,96%"
2,MNSA3,0.42,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
3,CSTB3,150.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"40,85%","28,98%",2.60,"22,40%","20,11%",0.0,8.420670e+09,0.14,"31,91%"
4,PORP4,2.40,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","-2,08%",0.0,2.239900e+07,0.00,"13,66%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,UBBR11,14.75,1201.81,3.91,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
980,UBBR3,18.00,1466.61,4.77,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
981,GLOB4,30.49,4873.60,8.59,1.512,"0,00%",1.389,-63.21,46.91,-4.59,55.03,28.77,"3,22%","0,03%",0.96,"3,90%","0,18%",0.0,5.675000e+09,1.76,"5,35%"
982,LWSA3,14.55,20328.00,2.91,12.352,"0,00%",2.041,5.08,272.27,8.30,218.45,63.18,"4,54%","0,06%",3.82,"1,33%","0,01%",166140000.0,2.950560e+09,0.04,"53,23%"


tratando os dados

In [7]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype('float')/100

In [8]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,PMET3,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0410,0.0,-2.908630e+08,0.00,0.3774
1,CLAN3,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,-0.0105,0.0,1.012240e+09,0.00,-0.6396
2,MNSA3,0.42,0.00,0.00,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,-2.0815,-3.6266,3.63,-0.1350,1.4570,0.0,-9.105000e+06,-6.52,-0.4111
3,CSTB3,150.00,0.00,0.00,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.4085,0.2898,2.60,0.2240,0.2011,0.0,8.420670e+09,0.14,0.3191
4,PORP4,2.40,0.00,0.00,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,-0.0208,0.0,2.239900e+07,0.00,0.1366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,UBBR11,14.75,1201.81,3.91,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0033,0.0,1.031720e+10,0.00,0.1058
980,UBBR3,18.00,1466.61,4.77,0.000,0.0,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0033,0.0,1.031720e+10,0.00,0.1058
981,GLOB4,30.49,4873.60,8.59,1.512,0.0,1.389,-63.21,46.91,-4.59,55.03,28.77,0.0322,0.0003,0.96,0.0390,0.0018,0.0,5.675000e+09,1.76,0.0535
982,LWSA3,14.55,20328.00,2.91,12.352,0.0,2.041,5.08,272.27,8.30,218.45,63.18,0.0454,0.0006,3.82,0.0133,0.0001,166140000.0,2.950560e+09,0.04,0.5323


**Analisando os dados**

empresa com liquidez 
1.000.000 média diária dos ultimos 2 meses

In [10]:
df = df[df['Liq.2meses'] > 1000000]
df.shape

(280, 21)

menor relacao EV/Ebit e
maior ROI

In [13]:
ranking = pd.DataFrame()
ranking['pos']=range(1,151)
ranking['EV/EBIT'] = df[ df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values
ranking.shape

(150, 3)

In [14]:
ranking

,pos,EV/EBIT,ROIC
0,1,PSSA3,CMIN3
1,2,GOAU3,WIZS3
2,3,GOAU4,PSSA3
3,4,EVEN3,TASA3
4,5,USIM3,TASA4
...,...,...,...
145,146,STBP3,DIRR3
146,147,LREN3,EZTC3
147,148,ENEV3,AMBP3
148,149,VIVA3,LAME3


In [15]:
a = ranking.pivot_table(columns='EV/EBIT', values='pos')

In [16]:
b = ranking.pivot_table(columns='ROIC', values='pos')

In [17]:
t = pd.concat([a,b])
t

,ABEV3,AGRO3,ALLD3,ALSO3,ALUP11,AMAR3,ASAI3,AURA33,B3SA3,BEEF3,BLAU3,BMOB3,BOAS3,BRFS3,BRKM3,BRKM5,BRML3,CAML3,CARD3,CCRO3,CESP6,CIEL3,CMIG3,CMIG4,CMIN3,CPFE3,CPLE11,CPLE3,CPLE6,CRFB3,CRPG5,CSED3,CSMG3,CSNA3,CURY3,CYRE3,DEXP3,DIRR3,DMMO3,DXCO3,...,TAEE3,TAEE4,TASA3,TASA4,TECN3,TEND3,TGMA3,TIMS3,TOTS3,TRIS3,TRPL4,TUPY3,UGPA3,UNIP3,UNIP6,USIM3,USIM5,VALE3,VBBR3,VIIA3,VITT3,VIVA3,VIVT3,VLID3,VULC3,WIZS3,AERI3,ALPA4,AMBP3,ARZZ3,GPIV33,INTB3,MOSI3,PETZ3,RADL3,RDOR3,RRRP3,TFCO4,VAMO3,WEGE3
pos,134,34,26,144.0,20,93,113,41,98,50,124,81,135,112,15,16,139.0,88,72,120,82,12,80,63,11,67,52,48,53,83,79,101,55,9,45,97,57,70,19,94,...,61,59,29,30,22,68,87,89,137,69,39,74,141,38,35,5,6,8,111,143.0,129,149,140.0,108.0,145,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pos,67,35,50,NaN,43,120,62,19,41,39,16,52,95,129,18,17,NaN,86,51,112,135,128,110,109,1,88,125,126,127,99,31,138,102,13,37,143,60,146,14,74,...,46,44,4,5,85,123,64,122,53,119,68,93,144,8,9,20,21,6,89,NaN,47,58,NaN,NaN,132,2,116.0,42.0,148.0,98.0,10.0,61.0,137.0,107.0,100.0,113.0,114.0,28.0,84.0,32.0


In [18]:
rank = t.dropna(axis=1).sum()
rank

ABEV3     201
AGRO3      69
ALLD3      76
ALUP11     63
AMAR3     213
         ... 
VBBR3     200
VITT3     176
VIVA3     207
VULC3     277
WIZS3      15
Length: 136, dtype: int64

In [19]:
rank.sort_values()[:15]
# retirar empresas do setor financeiro

PSSA3     4
CMIN3    12
VALE3    14
WIZS3    15
PLPL3    18
CSNA3    22
USIM3    25
ETER3    25
MRFG3    26
GOAU3    27
USIM5    27
GOAU4    29
TASA3    33
DMMO3    33
BRKM3    33
dtype: int64